In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

In [25]:

#Criando uma session com acesso ao gcp
spark = (
    SparkSession
    .builder
    .appName("Desafio-EDC-Extracao")
    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)
"""
    .config("spark.jars","./jars/gcs-connector-hadoop3-latest.jar")
    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    .config("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    .config("fs.gs.auth.service.account.enable","true")
    .config("fs.gs.auth.service.account.json.keyfile", "/mnt/d/EngDados/gcp/gcp-estudos-engdados-20f2cdfffed8.json")
    .config("spark.driver.memory", "8g")
)"""

'\n    .config("spark.jars","./jars/gcs-connector-hadoop3-latest.jar")\n    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\n    .config("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\n    .config("fs.gs.auth.service.account.enable","true")\n    .config("fs.gs.auth.service.account.json.keyfile", "/mnt/d/EngDados/gcp/gcp-estudos-engdados-20f2cdfffed8.json")\n    .config("spark.driver.memory", "8g")\n)'

## Leitura dos dados de CNAE - Atividade Economica

In [26]:
schema_cnae = (
    StructType([
        StructField("cod_atividade", StringType(),True ),
        StructField("desc_atividade", StringType(), True)
        ])
    )

In [27]:
df_cnae = spark.read.csv("gs://desafio-final/F.K03200$Z.D10710.CNAE.csv",
                         sep=";",
                         header=False,
                         encoding="ISO-8859-1",
                         escape="\"",
                         schema=schema_cnae)

In [28]:
df_cnae.limit(5).toPandas()

,cod_atividade,desc_atividade
0,0111301,Cultivo de arroz
1,0111302,Cultivo de milho
2,0111303,Cultivo de trigo
3,0111399,Cultivo de outros cereais não especificados an...
4,0112101,Cultivo de algodão herbáceo


In [29]:
df_cnae.write.mode("overwrite").parquet("gs://bootcamp-edc/raw/cnae.parquet")

In [30]:
del(df_cnae)

## Leitura dos dados de Município

In [31]:
schema_municipio = (
    StructType([
        StructField("cod_municipio", StringType(), True),
        StructField("desc_municipio", StringType(), True)
        ])
    )

In [32]:
df_municipio = spark.read.csv("gs://desafio-final/F.K03200$Z.D10710.MUNIC.csv",
                              sep=";",
                              header=False,
                              encoding="ISO-8859-1",
                              escape="\"",
                              schema=schema_municipio)

In [33]:
df_municipio.limit(5).toPandas()

,cod_municipio,desc_municipio
0,0001,GUAJARA-MIRIM
1,0002,ALTO ALEGRE DOS PARECIS
2,0003,PORTO VELHO
3,0004,BURITIS
4,0005,JI-PARANA


In [34]:
df_municipio.write.mode("overwrite").parquet("gs://bootcamp-edc/raw/municipio.parquet")

In [35]:
del(df_municipio)

## Leitura dos dados de estabelecimentos

In [36]:
schema_estabelecimento = (
    StructType([
        StructField("cnpj_basico", StringType(), True),
        StructField("cnpj_ordem", StringType(), True),
        StructField("cnpj_dig_verificador", StringType(), True),
        StructField("idc_matriz_filial", StringType(), True),
        StructField("nome_fantasia", StringType(), True),
        StructField("cod_situacao_cadastral", StringType(), True),
        StructField("data_situacao_cadastral", StringType(), True),
        StructField("motivo_situacao_cadastral", StringType(), True),
        StructField("nome_cidade_exterior", StringType(), True),
        StructField("cod_pais", StringType(), True),
        StructField("data_inicio_atividade", StringType(), True),
        StructField("cnae_fiscal_principal", StringType(), True),
        StructField("cnae_fiscal_secundaria", StringType(), True),
        StructField("tipo_logradouro", StringType(), True),
        StructField("nome_logradouro", StringType(), True),
        StructField("numero_logradouro", StringType(), True),
        StructField("complemento_logradouro", StringType(), True),
        StructField("nome_bairro", StringType(), True),
        StructField("cod_cep", StringType(), True),
        StructField("sigla_uf", StringType(), True),
        StructField("cod_municipio", StringType(), True),
        StructField("ddd_telefone_1", StringType(), True),
        StructField("num_telefone_1", StringType(), True),
        StructField("ddd_telefone_2", StringType(), True),
        StructField("num_telefone_2", StringType(), True),
        StructField("ddd_fax", StringType(), True),
        StructField("num_fax_2", StringType(), True),
        StructField("email", StringType(), True),
        StructField("situacao_especial", StringType(), True),
        StructField("data_situacao_especial", StringType(), True)
    ])
)

In [37]:
df_estabelecimento = spark.read.csv(
    "gs://desafio-final/estabelecimentos/*.csv",
    sep=";",
    header=False,
    encoding="ISO-8859-1",
    escape="\"",
    schema=schema_estabelecimento
)

In [38]:
df_estabelecimento.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,cod_pais,...,cod_municipio,ddd_telefone_1,num_telefone_1,ddd_telefone_2,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial
0,26638409,0001,21,1,CANTINHO DA LILICA,02,20161130,00,None,None,...,5425,31,92667948,None,None,None,None,elissouzap@hotmail.com,None,None
1,26638420,0001,91,1,NOBRE MADEIRAS DO BRASIL,04,20210212,63,None,None,...,9373,62,39881414,None,None,None,None,fmix.contabilidade@gmail.com,None,None
2,26638434,0001,05,1,TECNODONTOMEDICAL,08,20161216,01,None,None,...,3105,79,98704380,None,None,None,None,alex28ndre@gmail.com,None,None
3,26638446,0001,30,1,None,04,20210303,63,None,None,...,0849,98,32649607,None,None,None,None,None,None,None
4,26638459,0001,09,1,None,02,20161130,00,None,None,...,7107,11,99999999,None,None,None,None,None,None,None


In [39]:
df_estabelecimento.write.mode("overwrite").parquet("gs://bootcamp-edc/raw/estabelecimento.parquet")

In [40]:
spark.stop()